# Data preparation

### General data preparation
The following splits the raw data set into training and testing data sets.

After loading *carabids.csv* as `carabid`, the first step is to vizualize the species abundance distribution. This will help us decide a good abundance threshold for which species will be included in the first round of model training. Additional species can be added or removed based on insights gained in the model evaluation process.

In [ ]:
carabid = read.csv(file.choose())

sptable = table(carabid$SpeciesName)
sptable = sort(sptable, decreasing = T)

par(mar = c(2.5, 4, 2, 2))
barplot(sptable, 
        ylab = "Abundance", 
        ylim = c(0,300), 
        names.arg = ""
       )
abline(h = 30, col = "red", lty = 2)
mtext("Species", side = 1, line = 1)

There appears to be an abundance dropoff at 30 specimens, so we will use that as our threshold. Any species below the threshold will be added to `spname`. 

We then subset `carabid` as `carabid_keep`, removing any species in `spname` and specimens with unknown species labels ('Carabidae sp.').

In [ ]:
'%!in%' = function(x,y)!('%in%'(x,y))

spname = names(which(sptable < 30))

carabid_keep = subset(carabid, SpeciesName %!in% c('Carabidae sp.',spname))

Next, we will split the data into a training and testing set at a ratio of 80:20. Setting the seed here makes the code more reproduceable so the exact same split can be repeated again in the future if needed.

When computing the sample size of the training dataset, we use `floor()` to round down to the nearest integer. Other rounding techniques can be used as well, what matters is that we get a whole integer value.

We then take a random sample from our dataset and use it as our training dataset, with the remaining data being our test dataset.

In [ ]:
fractionTraining = 0.8
fractionvalid = 0.2

seed = 123
set.seed(seed)

# Compute sample sizes.
sampleSizeTraining = floor(fractionTraining * nrow(carabid_keep))

# Create the randomly-sampled indices for the dataframe. Use setdiff() to
# avoid overlapping subsets of indices.
indicesTraining = sort(sample(seq_len(nrow(carabid_keep)), size=sampleSizeTraining))
indicestest = setdiff(seq_len(nrow(carabid_keep)), indicesTraining)

dfTraining = carabid_keep[indicesTraining, ]
dfTest = carabid_keep[indicestest, ]

Now that we've split our data, we need to do some additional cleaning. First, we will remove unnecessary columns and confirm our predictor variables are numeric. Unnecessary columns will be any columns that do not contain class labels or numeric feature data that will be used to train the model. Our data contains some missing values, so a warning will appear stating that 'NAs introduced by coercion'. The algorithms we are training can handle NAs, so we can ignore that message.

In [ ]:
#Remove unnecessary columns
remove = c(1:17,25,29:32,43,47:50,55:57,65,73,81,89,97,105)
dfTraining = dfTraining[,-remove]
dfTest = dfTest[,-remove]

#Make predictor variables numeric
numcols = ncol(dfTraining)
dfTraining[,2:numcols] = as.data.frame(lapply(dfTraining[,2:numcols], as.numeric))
dfTest[,2:numcols] = as.data.frame(lapply(dfTest[,2:numcols], as.numeric))

Finally, we will standardize our data using the `preProcess` function from the `caret` package. This prevents features with larger magnitudes from dominating the learning process and negatively impacting the model's performance.

In [ ]:
library(caret)

normParam = preProcess(dfTraining)
norm.train = predict(normParam, dfTraining)
norm.test = predict(normParam, dfTest)

trainLabels = norm.train$SpeciesName

In [ ]:
testLabels = norm.test$SpeciesName

# Model Training

### XGBoost
The following code will train an eXtreme Gradient Boosting (XGBoost) model.

Before we begin, we must load the `xgboost` R package.

In [ ]:
library(xgboost)

The XGBoost model requires labels to be numeric starting at 0. The training data must also be reformatted as an `xgb.DMatrix` object to be compatible with the model. You can also format the test data as an `xgb.DMatrix` and include it as part of the `watchlist` object. This will allow you to monitor the performance of the model on the test data while the model trains, which will allow you to determine if/when the model becomes overfit.

In [ ]:
trainlab = as.numeric(as.factor(trainLabels)) - 1
testlab = as.numeric(as.factor(testLabels)) - 1

dtrain <- xgb.DMatrix(label = trainlab, data = as.matrix(norm.train[,2:69]))
dtest <- xgb.DMatrix(label = testlab, data = as.matrix(norm.test[,2:69]))
watchlist = list(train = dtrain, test = dtest)

Finally you can train your XGBoost model. A brief description of each of the model's parameters used here is as follows:

`data`: Your `xgb.DMatrix` training data.

`params`: A list of parameters. The parameters used here are:

- `max.depth`: The maximum complexity of the model's trees. The higher this number is, the more variables each tree will consider, which allows the model to capture more complex interactions between features. However, setting the value too high can lead to overfitting.

- `eta`: The learning rate of the model (min approaches 0, max = 1). A smaller eta value makes the boosting process more conservative by reducing the influence of each individual tree, preventing overfitting, but requiring more iterations (`nrounds`) to converge.

- `num_class`: The number of 'classes' (i.e. species) in the model.

- `eval.metric`: The evaluation performance metric ("mlogloss" [i.e. loss] or "merror" [i.e. 1 - accuracy]).

- `objective`: The objective function. Setting this as "multi:softprob" will give us a classification probability matrix when we classify new data with this model. Using "multi:softmax" would only give us the top-1 classifications without probabilities.

`watchlist`: Named datasets for model evaluation.

`nrounds`: The number of boosting iterations in the model. Increase this as you decrease `eta`.

`early_stopping_rounds`: Stops training if the performance (`eval.metric`) on a validation set does not improve for a specified number of consecutive rounds.

`verbose`: The amount of information that will be printed as the model trains. 0 = no information, 1 = some evaluation metrics (`eval.metric`), 2 = more evaluation metrics.

In [ ]:
num_class = 25
params = list(max.depth = 9,
              eta = 0.1,
              num_class = num_class,
              eval.metric = "mlogloss",
              objective = "multi:softprob")
model = xgb.train(data = dtrain,
                  params = params,
                  watchlist = watchlist,
                  nrounds = 250,
                  early_stopping_rounds = 25,
                  verbose = 1)

A learning curve can be plotting using the model's evaluation log.

In [ ]:
logloss_train = model$evaluation_log$train_mlogloss
logloss_test = model$evaluation_log$test_mlogloss

# Plot the logloss learning curve
plot(x = 1:length(logloss_train), 
     y = logloss_train, 
     type = "l", 
     col = "blue",
     xlab = "Number of Boosting Rounds", 
     ylab = "Logloss",
     main = "Logloss Learning Curve")
lines(x = 1:length(logloss_test), 
      y = logloss_test, 
      type = "l", 
      col = "red")
legend("topright", 
       legend = c("Training", "Testing"),
       col = c("blue", "red"), 
       lty = 1)

You can then make classifications using your model.

In [ ]:
probs = predict(model, as.matrix(norm.test[,2:ncol(norm.test)]))
probs = matrix(probs, nrow = num_class)
probs = t(probs)
probs = as.data.frame(probs)

### LightGBM

The following code will train a Light Gradient Boosting Machine (LightGBM).

To begin, we must load the `lightgbm` R package.

In [ ]:
library(lightgbm)

Like the XGBoost model above, the classification labels must be numeric and the data must be reformatted as an lgb.Dataset object.

In [ ]:
dtrain = lgb.Dataset(as.matrix(norm.train[,-1]), label = as.numeric(as.factor(norm.train[,1])) - 1)
dtest = lgb.Dataset(as.matrix(norm.test[,-1]), label = as.numeric(as.factor(norm.test[,1])) - 1)

valids = list(train = dtrain,
              test = dtest)

The lightGBM model is now ready to be trained. A brief description of the model's parameters are as follows:

`params`: A list of parameters. The parameters used here are:

- `objective`: Specifies the loss function to be optimized during the training of the gradient boosting model (e.g. 'regression', 'binary', 'multiclass').

- `metric`: Metrics to be evaluated on the evaluation set(s).

- `num_class`: The number of 'classes' (i.e. species) in the model.

- `learning_rate`: The step size learning rate of the model (same as `eta` in XGBoost).

- `min_data_in_leaf`: The minimum data used in one leaf.

- `num_leaves`: The maximum number of leaves allowed in a tree.

- `max_depth`: Maximum depth of a LightGBM tree.
    
`data`: a lgb.Dataset object used for training.

`nrounds`: number of training rounds.

`valids`: a list of lgb.Dataset objects used for validation

`verbose`: verbosity for output.

`early_stopping_rounds`: Activates early stopping. When this parameter is non-null, training will stop if the evaluation of any metric on any validation set fails to improve for `early_stopping_rounds` consecutive boosting rounds. If training stops early, the returned model will have attribute `best_iter` set to the iteration number of the best iteration.

For more information on LightGBM parameter tuning (particularly for the `num_leaves`, `min_data_in_leaf`, and `max_depth` parameters), see the LightGBM parameter tuning documenation https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html.

In [ ]:
num_class = 25
params = list(objective = 'multiclass',
              metric = 'multi_logloss',
              num_class = num_class,
              learning_rate = 0.1,
              min_data_in_leaf = 75,
              num_leaves = 40,
              max_depth = 15)

model = lgb.train(params = params,
                  data = dtrain,
                  nrounds = 100,
                  valids = valids,
                  verbose = -1,
                  early_stopping_rounds = 20)

A learning curve can be plotting using the model's evaluation log.

In [ ]:
logloss_train = lgb.get.eval.result(model, "train", "multi_logloss")
logloss_test = lgb.get.eval.result(model, "test", "multi_logloss")

# Plot the logloss learning curve
plot(x = 1:length(logloss_train), 
     y = logloss_train, 
     type = "l", 
     col = "blue",
     xlab = "Number of Boosting Rounds", 
     ylab = "Logloss",
     main = "Logloss Learning Curve")
lines(x = 1:length(logloss_test), 
      y = logloss_test, 
      type = "l", 
      col = "red")
legend("topright", 
       legend = c("Training", "Testing"),
       col = c("blue", "red"), 
       lty = 1)

You can then make classifications using your model.

In [ ]:
probs = predict(model, as.matrix(norm.test[,2:ncol(norm.test)]))
probs = matrix(probs, nrow = num_class)
probs = t(probs)
probs = as.data.frame(probs)

### Random Forest

The following code will train a Random Forest classification model.

To start, we must load the `randomForest` R package.

In [ ]:
library(randomForest)

Compared to the previous models, the training procedure for random forests is simple. The only data preparation step we need to do is ensure the class variable ('SpeciesName') is a factor. We can also use `na.roughfix` to fill any missing values in the test data. This allows predictions to be made on all the test specimens.

A brief description of the model's parameters are as follows:
`formula`: A formula describing the model to be fitted.

`data`: A data frame containing the variables in the model.

`ntree`: Number of trees to grow.

`mtry`: Number of variables randomly sampled as candidates at each split. Higher mtry values can increase the model's capacity to capture intricate relationships in the data, but can also reduce the diversity of the trees in the random forest if mtry approaches the total number of features in the dataset. Conversely, a low mtry can prevent overfitting, but might reduce the model's overall predictive power.

`na.action`: A function to specify the action to be taken if NAs are found.

`importance`: Should the performance of the predictors be assessed?

While random forest models are not inherently iterative, we can iterate over values of mtry using a training loop to find an optimal value. `mtry_range` determines the maximum range of mtry values that will be used. `early_stopping` determines when to stop iterating over different values of mtry based on the difference between the current mtry value and the best mtry. The model with the optimal value of mtry is saved as `best_model`.

In [ ]:
library(MLmetrics)

#Prepare data
norm.train$SpeciesName = as.factor(norm.train$SpeciesName)
norm.test$SpeciesName = as.factor(norm.test$SpeciesName)
norm.test = na.roughfix(norm.test)

#Initialize variables for the training loop. 
mtry_range = 5:50
early_stopping = 5
best_loss = Inf
best_mtry = mtry_range[1]

#Training loop
for(mtry in mtry_range){
  #Set seed for reproduceability
  set.seed(123)
    
  #Train model
  rf_model = randomForest(SpeciesName ~ ., 
                          data = norm.train, 
                          ntree = 500, 
                          mtry = mtry, 
                          na.action = na.roughfix, 
                          importance = TRUE)
    
  #Measure evaluation metrics
  probs = predict(rf_model, norm.test, type = "prob")
  colnames(probs) = levels(as.factor(testLabels))
  preds = unlist(apply(probs, MARGIN = 1, FUN = function(x){names(which.max(x))}))
  accuracy = mean(preds == norm.test$SpeciesName)
  loss = MultiLogLoss(probs, testLabels)
    
  #Report evaluation metrics
  print(paste("mtry =", mtry, "Accuracy =", signif(accuracy, 5), "Loss =", signif(loss,5)))
  
  #If loss has been improved over previous best, save new values for the listed variables
  if(loss < best_loss){
    best_loss = loss
    best_mtry = mtry
    best_model = rf_model
  }
    
  #If performance has not improved in [early_stopping] iterations, break the loop
  if((mtry - best_mtry) >= early_stopping){
    break
  }
}

You can then make classifications using your model.

In [ ]:
probs = predict(best_model, norm.test, type = "prob")

# Data evaluation

### Basic metrics


Most basic performance metrics for your model can be easily measured using the `confusionMatrix` function from the `caret` package.

To make our data compatible with the `confusionMatrix` function, we will need to convert our probability matrix to a vector of predicted classes.

In [ ]:
library(caret)
colnames(probs) = levels(as.factor(testLabels))
preds = unlist(apply(probs, MARGIN = 1, FUN = function(x){names(which.max(x))}))
confmat = confusionMatrix(as.factor(preds), as.factor(testLabels))

Measuring loss can easily be done using the `MLmetrics` package.

In [ ]:
library(MLmetrics)
loss = MultiLogLoss(probs, testLabels)

Accuracy and related metrics can be found in `confmat$overall`.

Many useful metrics for comparing performance within classes can be found in `confmat$byclass`, such as precision, recall, and F1 score. You can also measure the macro average of these metrics by taking measuring the average across all classes.

In [ ]:
accuracy = confmat$overall[1]
#Simply using mean() to measure macro averages can work, but might
#give an incorrect result if values are missing
f1 = sum(confmat$byClass[,"F1"], na.rm = T)/nrow(confmat$byClass)

The relationship between classification performance and species prevalence in the data can be visualized using data in `confmat$byclass`. Please note this assumes the relative species prevalence is the same in the training and testing datasets.

In [ ]:
Prevalence = confmat$byClass[,"Prevalence"]
F1 = confmat$byClass[,"F1"]

f1_lm = lm(F1~Prevalence)
plot(F1~Prevalence)
abline(f1_lm)

Our `confmat` object also contains a confusion matrix that we can plot using `ggplot2`. 

First, the confusion matrix must be reformated using the `melt` function from `reshape2`.

Then, we transform the values in the confusion matrix to be the proportion of the total number of true observations for each species. We do this using a custom `prop` function and functions within the `dplyr` package. This will allow our plot to be shaded proportionally for each species.

The resulting plot can be plotted normally, or saved as an interactive .html file using `plotly` & `htmlwidgets`.

In [ ]:
library(reshape2)
library(dplyr)
library(stringr)
library(ggplot2)
library(plotly)
library(htmlwidgets)

conf_df = as.data.frame(confmat$table)
conf_df = melt(conf_df)

prop = function(x){
  x/sum(x)
}

conf_df = conf_df %>%
  group_by(Reference) %>%
  mutate(prop = prop(value))

#Creating an ordered vectors of shortened species names
short_names = ordered(levels(factor(conf_df$Reference)))
short_names = str_replace(short_names, "(\\w)\\w+\\s(\\w+)(\\s\\w+)?", "\\1. \\2")

gg = ggplot(conf_df, aes(x = Prediction, y = Reference, fill = prop)) +
  geom_tile() +
  scale_fill_gradient(low = "#201547", high = "#00BCE1", name = "Proportion") +
  scale_x_discrete(labels = short_names, position = "top") +
  labs(y = "Actual") +
  annotate("text", x = 12.5, y = -0.5, label = "Predicted", hjust = 0.5, vjust = 0) +
  theme(axis.text.x = element_text(angle = 30, hjust = 0, vjust = 1),
        axis.title.x = element_blank(),
        panel.grid = element_blank(),
        panel.background = element_rect(fill = "white"))

p = ggplotly(gg)
p = p %>% layout(xaxis = list(side = "top"))

saveWidget(p, "confmat.html")

### Top x accuracy

If you want to measure your model's accuracy if it had several attempts to make a classification, you can use these custom functions. `topxacc` will return your models accuracy when given 'x' number of attempts, while `topxpreds` returns the top 'x' most likely classifications for your test specimens. (e.g. top 3 accuracy or top 3 predictions).

These functions work by finding the top 'x' highest probability classes for any given classification in your classification probability matrix.

In [ ]:
topxacc = function(x, testlab, prob){
  #Creating a dataframe of top 'x' predictions
  topxpreds = data.frame(matrix(NA, nrow = nrow(prob), ncol = x))
  for(i in 1:nrow(prob)){
    #Each row is reordered by descending probability. The name of the top 'x' columns
    #from each row are recorded in topxpreds
    topxpreds[i,] =  names(prob[i,order(as.numeric(prob[i,]), decreasing = T)])[1:x]
  }
  topxaccuracy = sum(testlab == topxpreds)/length(testlab)
  
  return(topxaccuracy)
}

topxpreds = function(x, testlab, prob){
  #Creating a dataframe of top 'x' predictions
  topxpreds = data.frame(matrix(NA, nrow = nrow(prob), ncol = x))
  for(i in 1:nrow(prob)){
    #Each row is reordered by descending probability. The name of the top 'x' columns
    #from each row are recorded in topxpreds
    topxpreds[i,] =  names(prob[i,order(as.numeric(prob[i,]), decreasing = T)])[1:x]
  }
  
  return(topxpreds)
}

top3acc = topxacc(3, testLabels, probs)
top3preds = topxpreds(3, testLabels, probs)

### ROC & AUC
Receiver Operating Characteristic (ROC) curves show the trade-off between true positive rate (sensitivity) and false positive rate (1-specificity) for varying classification thresholds. ROC curves are typically used in binary classification problems, but can be adapted to multi-class classification problems using a one vs. rest approach, where only one class is treated as the positive class and the remaining are treated as negative classes. This is repeated for each class in the dataset, resulting in an ROC curve for each class.


The AUC (Area Under the Curve) of an ROC curve quantifies the overall performance of a binary classification model by measuring the probability that a randomly chosen positive instance will be ranked higher than a randomly chosen negative instance according to the model's predicted probabilities. A higher AUC value indicates better discrimination and predictive power of the model.

In [ ]:
num_classes = 25
testlab = as.numeric(as.factor(testLabels)) - 1

roc_objs = lapply(1:num_classes, function(class_index) {
  roc_obj = roc(testlab == class_index, probs[, class_index])
  roc_obj
})
auc_values = sapply(roc_objs, function(roc_obj) auc(roc_obj))

To print any individual ROC curve, you can use the following code:

In [ ]:
#Replace i with whichever class index you want to plot an ROC curve for
i = 1
plot(roc_objs[[i]], main = paste("ROC Curve - Class", i), print.auc = TRUE)

### Hierarchical classification

Hierarchical classifiers allow you to make classifications at multiple taxonomic levels simultaneously. To set up a hierarchical classifier, you will need to make a taxonomic hierarchy dataframe for your data. First, create a vector called `hierarchylevels` that contains the column names in your dataset of the taxonomic levels you want to include. Then, use this custom `hierarchy` function to create your taxonomic hierarchy dataframe.

In [ ]:
hierarchylevels = c("SpeciesName", 
                    "Genus", 
                    "Subtribe", 
                    "Tribe", 
                    "Supertribe", 
                    "Subfamily")

hierarchy = function(data, ranks){
  
  baselabels = data[,ranks[1]]
  
  #Get all unique base labels
  uniquelabels = levels(as.factor(baselabels))
  
  #Create hierarchy DF
  hierarchy = data.frame(matrix(NA, nrow = length(uniquelabels), ncol = length(ranks)))
  colnames(hierarchy) = ranks
  #Set first column of DF as the unique baselabels
  hierarchy[,1] = uniquelabels
  #For loop starting with second taxonomic level/column
  for(i in 2:ncol(hierarchy)){
    #Iterate through each unique LITL label
    for(j in 1:length(uniquelabels)){
      row = which(baselabels == uniquelabels[j])[1]
      hierarchy[j,i] = as.character(data[row,ranks[i]])
    }
  }
  return(hierarchy)
}

myhierarchy = hierarchy(carabid, hierarchylevels)

Once you have your hierarchy dataframe, you can use it as a reference to convert your model's classifications into hierarchical classifications using this custom `hpredict` function. You can also use `hpredict` on your test labels for easier comparison with with your hierarchical classifications. You can then measure performance metrics at any taxonomic level using `confusionMatrix`.

In [ ]:
preds = top3preds[,1]

hpredict = function(preds, hierarchy){
  #Initializing prediction dataframe
  preddf = data.frame(matrix(NA, nrow = length(preds), ncol = ncol(hierarchy)))
  colnames(preddf) = colnames(hierarchy)
  preddf[,1] = preds
  
  
  for(i in 2:ncol(hierarchy)){
    for(j in 1:nrow(preddf)){
      #For the current pred (in this case preddf[j,i-1]), find the first match
      #in the hierarchy dataframe and assign the label in the next column up
      #as preddf[j,i].
      preddf[j,i] = hierarchy[which(hierarchy[,i-1] == preddf[j,i-1])[1], i]
    }
  }  
  return(preddf)
}

hpreds = hpredict(preds, myhierarchy)
htest = hpredict(testLabels, myhierarchy)

genusconfmat = confusionMatrix(as.factor(hpreds$Genus), as.factor(htest$Genus))
